In [0]:
CREATE OR REPLACE TABLE error_log (
  id BIGINT GENERATED ALWAYS AS IDENTITY,
  error_message STRING,
  sqlstate STRING,
  fail_code STRING
);

In [0]:
SELECT * FROM error_log ORDER BY id DESC LIMIT 1;

In [0]:
BEGIN
    -- Section 1: Variable Declarations
    DECLARE user_id INT DEFAULT 2;
    DECLARE user_count BIGINT;
    
    -- Section 2: Error Handling
    -- General error handler for any SQL exceptions
    DECLARE EXIT HANDLER FOR SQLEXCEPTION
    BEGIN
        -- Capture error details
        DECLARE error_state STRING;
        DECLARE error_msg   STRING;
        DECLARE error_cond  STRING;
        GET DIAGNOSTICS CONDITION 1
            error_cond  = CONDITION_IDENTIFIER,
            error_state = RETURNED_SQLSTATE,
            error_msg   = MESSAGE_TEXT;
        -- Log the error in error_log table
        INSERT INTO error_log(error_message, sqlstate, fail_code)
        VALUES (
            error_msg,
            error_state,
            error_cond
        );
    END;
    
    -- Section 3: Quality Check
    -- Fail if user_id is not greater than 0
    IF user_id <= 0 THEN 
        SIGNAL SQLSTATE '45001'
        SET MESSAGE_TEXT = 'Invalid user_id: must be > 0';
    END IF;
    
    -- Section 4: User Existence Check
    -- Check if the user exists in the users table
    SET user_count = (SELECT COUNT(*) 
    FROM users
    WHERE id = user_id);
    
    -- Section 5: User Insertion or Error Signaling
    -- Insert user if not exists, otherwise log and fail
    IF user_count = 0 THEN
        INSERT INTO users(id, name) 
        VALUES (user_id, 'New User');
    ELSE 
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'User already existed';
    END IF;
    
END;


In [0]:
CREATE TABLE inventory (
  item_id INT,
  quantity INT,
  last_update TIMESTAMP
);

In [0]:
SELECT * FROM inventory;

In [0]:
BEGIN
    DECLARE VARIABLE cnt INT;
    -- Check if the record exists
    SET cnt = (SELECT COUNT(*) FROM inventory WHERE item_id = 1001);
    
    IF cnt = 0 THEN 
       -- Insert new record if not exists
       INSERT INTO inventory (item_id, quantity, last_update)
       VALUES (1001, 50, current_timestamp());
    ELSE 
       -- If exists, update the quantity instead
       UPDATE inventory
       SET quantity = quantity + 50, last_update = current_timestamp()
       WHERE item_id = 1001;
    END IF;
END;

In [0]:
CREATE TABLE raw_source (
  id INT,
  col1 STRING,
  col2 STRING,
  load_date DATE
);

CREATE TABLE final_table (
  id INT,
  col1 STRING,
  col2 STRING
);

In [0]:
BEGIN
    DECLARE VARIABLE staging_table_name STRING;
    SET staging_table_name = CONCAT('staging_table_', CAST(UNIX_TIMESTAMP() AS STRING));
    
    CREATE OR REPLACE TABLE IDENTIFIER(staging_table_name) (
        id INT,
        col1 STRING,
        col2 STRING,
        load_date DATE
    );
    -- Step 1: Load data into staging table
    INSERT INTO IDENTIFIER(staging_table_name)
    SELECT * 
    FROM raw_source WHERE load_date = current_date();
    
    -- Step 2: Remove duplicates in staging (data cleansing)
    DELETE FROM IDENTIFIER(staging_table_name)
    WHERE id IN (
        SELECT id FROM IDENTIFIER(staging_table_name) GROUP BY id HAVING COUNT(*) > 1
    );
    
    -- Step 3: Transform and insert into final table
    INSERT INTO final_table (id, col1, col2)
    SELECT id, UPPER(col1) AS col1, col2
    FROM IDENTIFIER(staging_table_name);
    
    -- Step 4: Cleanup staging
    DROP TABLE IDENTIFIER(staging_table_name);
END;

In [0]:
BEGIN
  DECLARE ZERO_DIVIDE CONDITION FOR SQLSTATE '22012';
  
  -- Specific handler for division by zero
  DECLARE EXIT HANDLER FOR ZERO_DIVIDE
  BEGIN
    VALUES ('Divide by zero encountered');
  END;

  -- Generic exception handler
  DECLARE EXIT HANDLER FOR SQLEXCEPTION
  BEGIN
    VALUES ('A general error occurred');
  END;

  SELECT 10 / 0;  -- Raises SQLSTATE '22012'
END;


In [0]:
BEGIN
  DECLARE EXIT HANDLER FOR SQLEXCEPTION
  BEGIN
    INSERT INTO error_log VALUES ('An error occurred');
  END;

  INSERT INTO table1 VALUES (1, 'ok'); 
  SELECT 10 / 0;     -- triggers an error
  INSERT INTO table1 VALUES (2, 'won’t run');
END;


In [0]:
BEGIN
  DECLARE user_id INT DEFAULT 42;
  DECLARE user_count BIGINT;

  SET user_count = (SELECT COUNT(*) FROM users WHERE id = user_id);

  IF user_count = 0 THEN
    INSERT INTO users VALUES (user_id, 'New user');
  ELSE
    -- pretend to update existing record
    SELECT 'User already exists. Skipping insert' AS message;
  END IF;
END;
